In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import fingernet

# Comando "mágico" para garantir que os plots apareçam no notebook
%matplotlib inline

In [ ]:
# --- Substitua pelo caminho correto para os pesos do seu modelo ---
WEIGHTS_PATH = "../models/released_version/Model.pth"
# ----------------------------------------------------------------

# Caminho para a imagem de exemplo.
# Se esta imagem não existir, uma imagem de teste será criada.
IMAGE_PATH = "/storage/standard/fingerprints/SD258/images/orig/000/01/sd258_000_11-01_template_bad.png"

# Exibe a imagem que será usada
print(f"Usando a imagem: {IMAGE_PATH}")
plt.imshow(Image.open(IMAGE_PATH), cmap='gray')
plt.title("Imagem de Entrada")
plt.axis('off')
plt.show()

In [ ]:
model = fingernet.get_fingernet(weights_path=WEIGHTS_PATH)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carrega a imagem, converte para tons de cinza e normaliza para [0, 1]
img_pil = Image.open(IMAGE_PATH).convert('L')
img_np = np.array(img_pil, dtype=np.float32) / 255.0

# Converte para tensor, adiciona as dimensões de lote e canal, e move para o dispositivo
input_tensor = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).to(device)

print(f"Formato do tensor de entrada: {input_tensor.shape}")
print(f"Dispositivo do tensor: {input_tensor.device}")

In [ ]:
results = model(input_tensor)

# Vamos inspecionar as chaves do dicionário de resultados
print("Chaves do dicionário de resultados:", results.keys())
print("\nNúmero de minúcias detectadas:", len(results['minutiae'][0]))

In [ ]:
# Prepara o dicionário de dados para a função de plotagem
# Move os tensores da GPU para a CPU e converte para NumPy
plot_data = {
    'input_path': IMAGE_PATH,
    'minutiae': results['minutiae'][0].cpu().numpy(),
    'enhanced_image': results['enhanced_image'][0].cpu().numpy(),
    'orientation_field': results['orientation_field'][0].cpu().numpy(),
    # A máscara não é usada por plot_output, então podemos omiti-la
}

# Gera e exibe o plot com todos os resultados
fingernet.plot_output(plot_data)

In [ ]:
#input_image = input_tensor.squeeze().cpu().numpy()
input_image = results['enhanced_image'][0].cpu().numpy()
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
fingernet.plot_input(ax, input_image)
fingernet.plot_mnt(ax, results['minutiae'][0].cpu().numpy())